# Trabajo Práctico Final - Web Mining

## Análisis de la oferta laboral en Argentina

**GRUPO 5: Alberto Gir Erdmann, Agostina Misaña, Carla Menzella, Ciro Couste, Verónica Fumero y Arnaldo Viera**

### ETAPA 1 - Generación del Dataset mediante herramientas de web-scraping:

Se aplica técnicas de web-scraping a través de la libreria Selenium a la página web de búsquedas laborales https://www.randstad.com.ar/, con el objetivo de obtener información acerca de la oferta laboral actualizada a Junio 2022, existente en todo el territorio de la Argentina. Se genera un dataframe(Job Title, Job Type, Job Description, Location) y se guarda en un .csv que luego se trabajará en la parte 2. 

In [1]:
import selenium
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import random
import time
from time import sleep
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_jobs(url):
    
    '''Gathers jobs as a dataframe, scraped from Randstad'''
    
    # Iniciar el webdriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(.1)
    
    # Seleccionar "Aceptar Cookies" 
    try:
        WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"/html/body/div[3]/div[2]/div/div[1]/div/div[2]/div/button[2]"))).click()
        time.sleep(.1)
    except:
        exit
    
    # Cantidad total de empleos encontrados
    inputElement = driver.find_element_by_css_selector("h1")
    inputelement = inputElement.text
    print("Cantidad total de empleos encontrados", inputElement.text)

    # Cantidad total encontrada en la pagina actual
    cantjobs=driver.find_element(By.XPATH, '//*[@id="search-results"]/div/div[2]/div[2]')
    print("Cantidad total de empleos cargados en estas paginas:" , cantjobs.text)
    inputelement2=cantjobs.text

    # Cargar todas las paginas
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="search-results"]/div/div[2]/button'))).click()
    while True:                                                             
        try:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="search-results"]/div/div[2]/button'))).click()
        except:
            break
    
    # Seleccionar a cada uno de los trabajos 
    cards=driver.find_elements_by_class_name('cards__link')
    cardsp=cards[:]
    
    # Establecer un conjunto con todas las pestañas que existen en este momento
    previous_tabs = set(driver.window_handles)

    # Pestaña actual
    previous_tab = driver.current_window_handle
    
    jobs=[]

    for card in cardsp:
        try:
            card.send_keys(Keys.CONTROL + Keys.RETURN)
            
            # Abrir la nueva pestaña haciendo click en el elemento
            WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(len(previous_tabs) + 1))
           
            # Obtener la nueva pestaña
            newWindow = (set(driver.window_handles) - previous_tabs).pop()
           
            # Cambiar el foco a la nueva pestaña
            driver.switch_to.window(newWindow)
            
            # Obtener los datos
            tit=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR,'h1'))).text
            place_time=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME,'cards__meta-item--link'))).text    
            description=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME,'basic-layout__main'))).text.split('\n')
            tipo, modo=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME,'meta-content__description'))).text.split('\n')[:2]    
            jobs.append({"Job Title" : tit,"Job Type": modo, "Job Description" : description,"Location" :place_time})
        
            # Cerrar la pestaña
            driver.close()
        
            # Volver a la pestaña inicial
            driver.switch_to.window(previous_tab)
            
            print("Scraping not terminated. Got:", len(jobs), end='\r')
            if len(jobs)==len(cardsp):
                print("Scraping terminated. Got {}.".format(len(jobs)))   
        except:
            break
    
    df=pd.DataFrame(jobs)
    df.to_csv('df_allthepositions.csv')
    return df

In [3]:
url='https://www.randstad.com.ar/trabajos/'
get_jobs(url)



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
There is no [win32] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\cmenzella\.wdm\drivers\chromedriver\win32\103.0.5060.53]


Cantidad total de empleos encontrados 767 trabajos encontrados para vos
Cantidad total de empleos cargados en estas paginas: 30 de 767 trabajos vistos
Scraping terminated. Got 767. 767


,Job Title,Job Type,Job Description,Location
0,comercial agro.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","necochea, buenos aires"
1,vendedor/ra de salón.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","san juan, san juan"
2,vendedor/ra.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","gualeguaychu, entre rios"
3,tax analyst.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","capital federal, capital federal"
4,operario/a taller de muestra.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","tortuguitas, buenos aires"
...,...,...,...,...
762,aparador/ra de calzados.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","perez, santa fe"
763,maestranza.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","maipu, mendoza"
764,maestranza /housekeeping.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","tunuyan, mendoza"
765,operario/a de bodega.,tiempo completo,"[¡Esta propuesta te puede interesar!, En Rands...","maipu, mendoza"
